<a href="https://colab.research.google.com/github/Remdeht/ia_detector/blob/master/Mapping_Irrigated_Areas_CampoDeCartagena.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Irrigated Area Detection for the Campo de Cartagena in Murcia, Spain**

###This notebook contains the code to generate a seasonal map of irrigated areas for the Campo de Cartagena in Murcia, Spain covering the summer and winter season over a one year period starting from 1985.
\
## Requirements

To be able to run this notebook you need to have a Google account with access to [**Google Drive**](https://www.google.com/drive/) and [**Google Earth Engine**](https://earthengine.google.com/). You can find more information on how to sign up for the Google Earth Engine [**here**](https://signup.earthengine.google.com).

\

---
---

***All information in this notebook is provided in good faith, however, we make no representation or warranty of any kind, express or implied, regarding the accuracy, adequacy, validity, reliability, availability, or completeness of any data presented in or derived from this notebook***

---
---


## **Before running the notebook we need to clone the Github repository, load the required python libraries and initiate your Earth Enngine account so you can access the GEE python API**

To run the Google Earth Engine Python API in this notebook, you need to complete an authentication process to verify that you have a access to the GEE. 

Run the code block below to start the authentication process. The code block's output will show a link in blue, which will lead you to a Notebook Authenticator page. Follow the steps on this page to generate the verfication code (detailed instructions can be found [**here**](https://developers.google.com/earth-engine/guides/python_install#expandable-1)), which you can then paste into the prompt (following the text "**Enter verification code:**") to finalize the authentication process.

After completing the authentication process, run the second code block to clone our github repository and install the Python libaries needed to run the code in this notebook.  

In [ ]:
#@title ## **Initiate the Google Earth Engine Python API**
#@markdown <font color='yellow'>**<== Run this code block to initiate the GEE Python API** </font>
from google.colab import output
import ee

try:
  ee.Initialize()
except:
  ee.Authenticate()  # if first time user of the Google Earth Engine on this device, otherwise ee.Initialize() is enough

output.clear()
print('Done!')

In [ ]:
#@title ## **Clone our Github repository and install Python libraries!**
#@markdown <font color='yellow'>**<== Run this block to clone our Github repository and install the Python libraries** </font>

!git clone https://github.com/Remdeht/ia_detector.git ia

!pip install monthdelta
!pip install folium==0.10.1

# Load libraries
import sys
import os
import glob
from datetime import datetime 

import bz2
import pickle
import _pickle as cPickle

from IPython.display import display

import folium
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import pandas as pd

# Load functions from the repo
from ia.gee_functions import GEE_USER_PATH, CALIBRATION_MAPS, CALIBRATION_LC_CLASSES, BANDNAMES, PALETTE_RF, PALETTE_IA
from ia.gee_functions.classification import classify_irrigated_areas, join_seasonal_irrigated_areas, create_feature_data, combine_training_areas
from ia.gee_functions.lda import take_strat_sample, remove_outliers, get_lda_params, perform_lda_scaling, get_data, get_histogram
from ia.gee_functions.export import track_task, export_to_drive, export_to_asset
from ia.gee_functions import visualization
from ia.gee_functions.validation import calc_area, calc_validation_score

aoi = ee.FeatureCollection(f'users/Postm087/vector/outline/outline_cdc')
aoi_coordinates = aoi.geometry().bounds().getInfo()['coordinates']
aoi_centroid = aoi.geometry().bounds().centroid(1).coordinates().getInfo()

output.clear()
print('Done!')

##**Some background Information bout the Campo de Cartagena**

The Campo de Cartagena area is located in the Murcia province in south-eastern Spain and has a semi-arid climate with an average yearly rainfall of 300mm and an average temperature of 18 degrees Celsius. The summers are typically dry and warm, while the winters are mild. In the period between 1988 and 2009, the Campo de Cartagena has undergone a dramatic change in land cover. Over time, natural vegetation made place for urban areas and (irrigated) herbaceous and tree crops as well as infrastructure related to irrigation, such as ponds for the storage of irrigation water. Crops grown within this region can be grouped into two main categories: seasonal herbaceous crops such as lettuce, broccoli and melon and perennial fruit tree crops, often carrying citrus fruits like oranges, lemons and mandarins. Irrigation for these crops is almost exclusively performed via drip irrigation.

**So why monitor agricultural water use?**

The agricultural sector consumes a large share of the total water supply, particularly in the Campo de Cartagena where irrigation agriculture has been steadily growing over the last decades. Besides leading to high water demands, irrigation agriculture can also have serious negative effects on the environment. A well-known case of negative environmental effects is the eutrophication is the Mar Menor, a saltwater lagoon located in the Campo de Cartagena. The widespread use of fertilizers in combination with irrigation and flash floods caused the levels of nitrogen, phosphorus and organic matter in the water to rise dramatically, causing an accelerated algal bloom disturbing the ecosystem's natural equilibrium. The subsequent increase in phytoplankton caused the oxygen levels in the water to drop, killing a large proportion of the fish population.

To prevent such a natural catastrophe from happening again it is important for the agricultural sector, managers and policymakers to handle water resources as efficiently as possible to meet food demands, while also keeping into account environmental factors. With this notebook, we hope to provide a tool that can help keeping an eye on irrigation practices in the Campo de Cartagena by allowing quick mapping of irrigated areas using relatively recent remote sensing data.

In [ ]:
#@title Run this code block to have a look at the Campo de Cartagena on the map
#@markdown <font color='yellow'>**<== Run this code block to check out our area of interst: the Campo de Cartagena** </font>

#@markdown **About the Map:** 
#@markdown To visualize the results on a map we are using [folium](https://python-visualization.github.io/folium/). 
#@markdown You can control the layers shown on the map using the layer control in the top left corner of the map window.
#@markdown If you can't see the layer control widget, try scrolling up using the scrollbar on the right side of your output cell.  

layer = {
    'Area of Interest':aoi.getMapId({'color':'FFFFFF'})
}

map = visualization.create_folium_map(layer, coords=[aoi_centroid[1], aoi_centroid[0]], zoom=11, height='100%')

folium.TileLayer(  # Add the Google Sattelite Map as a Basemap
        tiles='http://mt1.google.com/vt/lyrs=h&x={x}&y={y}&z={z}',
        attr='Google',
        name='Google Sat',
        overlay=False,
        control=False,
        max_zoom=20
    ).add_to(map)

display(map)

# **Classification**

With the GEE API and cloning the Github repository set up, you can start the classification process. The entire process consists of 4 steps, with each step generating a raster output that will be stored as a GEE asset (roughly 4GB in total). Export tasks may take quite some time to complete, you can check the status of your task or cancel your tasks here: [https://code.earthengine.google.com/tasks](https://code.earthengine.google.com/tasks).





#**Step 1 - Generate the data**

The first step is to select the year for which you want to generate the irrigated area map. In the code block below there is a slider that you can use to select the year that you want, starting in 1985. Based on your selection, two spectral datasets will be created, one with data for the "summer" season  (starting on the first of April of your year of choice up to the last day of September) and one with data for the "winter" season (starting on the first of October of your year of choice and ending at the 31st of March of the consecutive year). To generate the these datasets we use Tier 1 surface reflectance imagery from the [Landsat 5](https://developers.google.com/earth-engine/datasets/catalog/landsat-5), [Landsat 7](https://developers.google.com/earth-engine/datasets/catalog/landsat-7), [Landsat 8](https://developers.google.com/earth-engine/datasets/catalog/landsat-8) and [Landsat 9](https://developers.google.com/earth-engine/datasets/catalog/landsat-9) libraries.

In [ ]:
#@title #**Select a year for which to perform the classification using the slider below**
#@markdown <font color='yellow'>**<== Don't forget to run this code block to generate the dataset. This may take a while to complete!** </font>
results_folder = f'{GEE_USER_PATH}/ia_classification/raster/results/'
year = 1985 #@param {type:"slider", min:1985, max:2021, step:1}

# Sensor param can be changed to "sentinel" (Sentinel 2 MSI).
# However, as we used Landsat data for the calibration, the training areas generated
# using sentinel 2 data might unreliable and you might have to play around a bit 
# with the thresholds for better results.  

sensor = "landsat"   

if sensor == 'sentinel':
  scale=10
else:
  scale=30

summer_dates = (f'{year}-04-01', f'{year}-09-30')
winter_dates = (f'{year}-10-01', f'{year + 1}-03-31')

tasks = {}

task = create_feature_data(
    summer_dates,
    aoi,
    aoi_name='cdc',
    sensor=sensor,
    custom_name=f'summer_{year}',
    overwrite=False,
)
tasks['summer'] = task

task = create_feature_data(
    winter_dates,
    aoi,
    aoi_name='cdc',
    sensor=sensor,
    custom_name=f'winter_{year}',
    overwrite=False,
)
tasks['winter'] = task

while not track_task(tasks):
  pass

output.clear()
print('The data for classification has been generated!')

feature_data_summer = ee.Image(
        f'{GEE_USER_PATH}/ia_classification/raster/data/cdc/{sensor}/all_scenes_reduced/feature_data_cdc_summer_{year}'
        )

feature_data_winter = ee.Image(
        f'{GEE_USER_PATH}/ia_classification/raster/data/cdc/{sensor}/all_scenes_reduced/feature_data_cdc_winter_{year}'
        )


images = {
    f'RGB Composite of derived from satellite data for the summer of {year}':feature_data_summer.clip(aoi).getMapId(visualization.vis_params_rgb(bands=['R_median', 'G_median', 'B_median'], max_val = 0.3)),
    f'RGB Composite of derived from satellite data for the winter of {year}':feature_data_winter.clip(aoi).getMapId(visualization.vis_params_rgb(bands=['R_median', 'G_median', 'B_median'], max_val = 0.3)),
}

from google.colab.output import eval_js
eval_js('google.colab.output.setIframeHeight("800")')

map = visualization.create_folium_map(images, coords=[aoi_centroid[1], aoi_centroid[0]], zoom=10, height='100%')
map

##**Step 2 - Generating training areas**

To generate training areas we use [LULC maps](https://zenodo.org/record/237068) for the Campo de Cartagena for the years 1997, 2000 and 2009. By combining the spectral information from Landsat 5 and 7 with the land cover information from the calibration maps 

#### **Linear  Discriminant  Analysis**

Linear  Discriminant  Analysis (LDA) is as statistical method that aims to enhance the separability between classes by looking for the highest ratio of between-class variance to within-class variance. We distinguished 8 different land cover classes based on the calibration dataset, and sampled Landsat data for each land cover classes for matching the timeperiod of the calibration datasets in 1997, 2000 and 2009. It is important to note that our goal is to detect irrigated areas and we are not interested in other land cover types. As a result we use spectral indices proposed especially for the detection of irrigated areas.  

This resulted in a calibration dataset containing spectral information for 8 land cover classes over time in the Campo de Cartagena. Next, we aim to use the information about land cover types from this calibration dataset to separate land cover areas for other years.

This is where the LDA comes into play. For each of our 8 land cover class, we use the LDA to find the transform of the calibration dataset that maximizes the separability of our target land cover class compared to all the other land cover classes. For each land cover class we set a threshold value that best separates the pixels belonging to the target class from the other pixels. The following two code blocks visualize the distribution these pixel values for each land cover class, as well as the threshold value.

Next, we apply the same transformation to the feature dataset of the year for which we want to generate a new land cover map. We then select the pixels above the threshold value, leaving us with pixels that based on their spectral information are likely to belong to a land cover class. This creates a collection of pixels for each land cover class, which we use to train a Random Forest Classifier.

In [ ]:
# @title **Load the calibration datasets**
# @markdown <font color='yellow'> <== **Run this cell to load the calibration datasets - This may take a few minutes** </font>
data_summer = bz2.BZ2File(glob.glob('/content/ia/data/lda_calibration_summer.pbz2')[0], 'rb')
data_winter = bz2.BZ2File(glob.glob('/content/ia/data/lda_calibration_winter.pbz2')[0], 'rb')

lda_parameters_summer = cPickle.load(data_summer)
lda_parameters_winter = cPickle.load(data_winter)

print('Calibration datasets loaded succesfully!')

In [ ]:
#@title **Thresholds Summer** {run:"auto"}
# @markdown <font color='yellow'> <== **Run this cell to visualize the distribution of calibration data after the LDA** </font>

# @markdown The figure contains one histogram showing the separation of the calibration data after performing the LDA. 
# @markdown Use the sliders below to adjust the value of the threshold value to use for the generation of the training areas. 

# @markdown ---

summer_irrigated_trees_threshold = 1 #@param {type:"slider", min:-3, max:3, step:0.01}
summer_irrigated_crops_threshold = 1.35 #@param {type:"slider", min:-3, max:3, step:0.01}
summer_forest_threshold = 1.5 #@param {type:"slider", min:-3, max:3, step:0.01}
summer_shrub_threshold = 0.75 #@param {type:"slider", min:-3, max:3, step:0.01}
summer_rainfed_agriculture_threshold = 1 #@param {type:"slider", min:-3, max:3, step:0.01}
summer_greenhouses_threshold = 1 #@param {type:"slider", min:-3, max:3, step:0.01}
summer_urban_fallow_threshold = 1 #@param {type:"slider", min:-3, max:3, step:0.01}
summer_water_bodies_threshold = 2 #@param {type:"slider", min:-3, max:3, step:0.01}

user_summer_thresholds = {
    'summer_irrigated_trees': summer_irrigated_trees_threshold,
    'summer_irrigated_crops': summer_irrigated_crops_threshold,
    'summer_forest': summer_forest_threshold,
    'summer_shrub': summer_shrub_threshold,
    'summer_rainfed_agriculture': summer_rainfed_agriculture_threshold,
    'summer_greenhouses': summer_greenhouses_threshold,
    'summer_urban_fallow': summer_urban_fallow_threshold,
    'summer_water_bodies': summer_water_bodies_threshold,
}

training_areas_summer = None

plot_position = {
    0:(1, 1),
    1:(1, 2),
    2:(2, 1),
    3:(2, 2),
    4:(3, 1),
    5:(3, 2),
    6:(4, 1),
    7:(4, 2),
}

fig = make_subplots(rows=4, cols=2, subplot_titles=tuple(lda_parameters_summer.keys()))

for ind, cl in enumerate(lda_parameters_summer):
  
  X_summer, y_summer, lda_intercept_summer, lda_coefficients_summer, X_lda_sklearn_summer, gt_summer = lda_parameters_summer[cl]
  
  _classes = {
      1: 'other',
      2: cl,
  }

  _classes_detailed = {}

  for pix_val in y_summer['lc'].unique():
    
    class_name = y_summer[y_summer['lc'] == pix_val]['class'].unique()[0]
    
    if class_name == cl.replace('summer_', ''):
      _classes_detailed[0] = {'name':class_name, 'lc_val':pix_val}
    else:
      _classes_detailed[int(pix_val)] = class_name
  
  suggested_threshold_summer = user_summer_thresholds[cl]

  ta_summer = perform_lda_scaling(
      feature_data_summer,
      lda_intercept_summer,
      lda_coefficients_summer, 
      suggested_threshold_summer, 
      gt=gt_summer
  )
  
  if training_areas_summer is None:
      training_areas_summer = ta_summer.select('training').rename(cl.replace('summer_', ''))
  else:
      training_areas_summer = training_areas_summer.addBands(ta_summer.select('training').rename(cl.replace('summer_', '')))

  fig = get_histogram(X_lda_sklearn_summer, y_summer['lc'], _classes_detailed, suggested_threshold_summer, fig=fig, row=plot_position[ind][0], col=plot_position[ind][1])

fig.update_layout(height=700, showlegend=True)
fig.show()


In [ ]:
#@title ## **Thresholds Winter**  {run:"auto"}
winter_irrigated_trees_threshold = 0.9  # @param {type:"slider", min:-3, max:3, step:0.01}
winter_irrigated_crops_threshold = 1.3  # @param {type:"slider", min:-3, max:3, step:0.01}
winter_forest_threshold = 1.7  # @param {type:"slider", min:-3, max:3, step:0.01}
winter_shrub_threshold = 0.95  # @param {type:"slider", min:-3, max:3, step:0.01}
winter_rainfed_agriculture_threshold = 0.9  # @param {type:"slider", min:-3, max:3, step:0.01}
winter_greenhouses_threshold = 1.4  # @param {type:"slider", min:-3, max:3, step:0.01}
winter_urban_fallow_threshold = 1  # @param {type:"slider", min:-3, max:3, step:0.01}
winter_water_bodies_threshold = 2  # @param {type:"slider", min:-3, max:3, step:0.01}

user_winter_thresholds = {
    'winter_irrigated_trees': winter_irrigated_trees_threshold,
    'winter_irrigated_crops': winter_irrigated_crops_threshold,
    'winter_forest': winter_forest_threshold,
    'winter_shrub': winter_shrub_threshold,
    'winter_rainfed_agriculture': winter_rainfed_agriculture_threshold,
    'winter_greenhouses': winter_greenhouses_threshold,
    'winter_urban_fallow': winter_urban_fallow_threshold,
    'winter_water_bodies': winter_water_bodies_threshold,
}

training_areas_winter = None

plot_position = {
    0: (1, 1),
    1: (1, 2),
    2: (2, 1),
    3: (2, 2),
    4: (3, 1),
    5: (3, 2),
    6: (4, 1),
    7: (4, 2),
}

fig = make_subplots(rows=4, cols=2, subplot_titles=tuple(lda_parameters_winter.keys()))

for ind, cl in enumerate(lda_parameters_winter):

    X_winter, y_winter, lda_intercept_winter, lda_coefficients_winter, X_lda_sklearn_winter, gt_winter = \
    lda_parameters_winter[cl]

    if gt_winter:
      _classes = {
          1: 'other',
          2: cl,
      }
    else:
      _classes = {
          1: cl,
          2: 'other',
      }

    _classes_detailed = {}
    
    for pix_val in y_winter['lc'].unique():
        
        class_name = y_winter[y_winter['lc'] == pix_val]['class'].unique()[0]
        
        if class_name == cl.replace('winter_', ''):
            _classes_detailed[0] = {'name':class_name, 'lc_val':pix_val}
        else:
            _classes_detailed[int(pix_val)] = class_name

    suggested_threshold_winter = user_winter_thresholds[cl]

    ta_winter = perform_lda_scaling(
        feature_data_winter,
        lda_intercept_winter,
        lda_coefficients_winter,
        suggested_threshold_winter,
        gt=gt_winter
    )

    if training_areas_winter is None:
      training_areas_winter = ta_winter.select('training').rename(cl.replace('winter_', ''))
    else:
      training_areas_winter = training_areas_winter.addBands(ta_winter.select('training').rename(cl.replace('winter_', '')))

    fig = get_histogram(X_lda_sklearn_winter, y_winter['lc'], _classes_detailed, suggested_threshold_winter, fig=fig,
                        row=plot_position[ind][0], col=plot_position[ind][1])

fig.update_layout(height=700, showlegend=True)
fig.show()


## Generating the training areas 

Now that the thresholds for each class have been set, the next step is to perform the LDA scaling for our feature dataset, and to select the pixels for each land cover class to use for the training of the Random Forest classifiers, based on the thresholds we've specified before. Run the code block below to generate the training areas and visualize them on a map.   

In [ ]:
# @markdown <font color='yellow'> <== **Run this code block to generate the training areas** </font>

lc_classes = {
    'irrigated_trees':1,
    'irrigated_crops':2,
    'forest':3,
    'shrub':4,
    'rainfed_agriculture':5,
    'greenhouses':6,
    'urban_fallow':7,
    'water_bodies':8,
}

tasks = {}

try:
  task_summer = export_to_asset(
              asset=combine_training_areas(training_areas_summer, lc_classes),
              asset_type='image',
              asset_id=f"results/random_forest/cdc/training/training_summer_{year}",
              region=aoi_coordinates,
              scale=scale,
              overwrite=True
          )
  tasks['training areas summer'] = task_summer
except FileExistsError as e:
  print(e)

try:
  task_winter = export_to_asset(
              asset=combine_training_areas(training_areas_winter, lc_classes),
              asset_type='image',
              asset_id=f"results/random_forest/cdc/training/training_winter_{year}",
              region=aoi_coordinates,
              scale=scale,
              overwrite=True
          )
  tasks['training areas winter'] = task_winter
except FileExistsError as e:
  print(e)

if len(tasks) > 0:
  track_task(tasks)

output.clear()
print('Training Areas generated!')

summer_training = ee.Image(f'{results_folder}random_forest/cdc/training/training_summer_{year}')
winter_training = ee.Image(f'{results_folder}random_forest/cdc/training/training_winter_{year}')

images = {
    f'RGB Composite of derived from satellite data for the summer of {year}':feature_data_summer.clip(aoi).getMapId(visualization.vis_params_rgb(bands=['R_median', 'G_median', 'B_median'], max_val= 0.3)),
    f'RGB Composite of derived from satellite data for the winter of {year}':feature_data_winter.clip(aoi).getMapId(visualization.vis_params_rgb(bands=['R_median', 'G_median', 'B_median'], max_val= 0.3)),
    f'Training Areas Summer':summer_training.clip(aoi).getMapId(visualization.vis_params_cp('constant', 0, 8, list(PALETTE_RF.values()))),
    f'Training Areas Winter':winter_training.clip(aoi).getMapId(visualization.vis_params_cp('constant', 0, 8, list(PALETTE_RF.values()))),      
}

from google.colab.output import eval_js
eval_js('google.colab.output.setIframeHeight("8000")')

map = visualization.create_folium_map(images, coords=[aoi_centroid[1], aoi_centroid[0]], zoom=12, height='100%')
map = visualization.create_categorical_legend(map, PALETTE_RF)
output.clear()
map 

# Step 3 - Classification

After generating the training areas, the next step is to run the Random Forest classification for the summer and winter season of the year of your choosing. 

You can adjust the parameters for the classifiers using the sliders in the next code cell. Once you have selected the parameters you want you can run the cell to perform the classification. The classification will take some time, depending on the number of trees selected and Google's servers. Running a classification with the default parameters takes around 10 minutes. 

When the classification has completed the output will show a confustion matrix, with an accuracy score. This score is based on the training pixels that could best be separated from other land cover classes using the LDA, so the confusion between classes is naturally very low. However, it does give an indication of potential errors in our Random Forest classification product, which is important to keep in mind.       

In [ ]:
# @markdown <font color='yellow'> <== **Run this code block to perform the Random Forest Classification** </font>
#@markdown ---
#@markdown ## Set the classification parameters & run the code block to start the classification
#@markdown ---
#@markdown ### Number of Trees
no_trees = 200 #@param {type:"slider", min:0, max:500, step:10}
#@markdown ### Variables per Split
vps = 6 #@param {type:"slider", min:2, max:10, step:1}
#@markdown ### Bagging Fraction
bf = 0.65 #@param {type:"slider", min:0, max:1, step:0.05}
#@markdown ### Minimum Number of Training Points
min_tp = 500 #@param {type:"slider", min:100, max:5000, step:100}
#@markdown ### Maximum Number of Training Points
max_tp = 7000 #@param {type:"slider", min:100, max:25000, step:100}
classification_tasks = {}

classification_data = {
    'summer':{
        'feature_data':feature_data_summer,
        'training_data':summer_training.rename('training').reproject(feature_data_summer.projection()),
        },
    'winter':{
        'feature_data':feature_data_winter,
        'training_data':winter_training.rename('training').reproject(feature_data_winter.projection()),
    }
}

classification_tasks = {}
classifiers = {}

for season in classification_data:
  training = classification_data[season]['training_data']
  training = training.addBands(training)

  classification_task, clf = classify_irrigated_areas(
        classification_data[season]['feature_data'],
        training,
        aoi,
        aoi_name='cdc',
        season=season,
        year=str(year),
        no_trees=no_trees,
        bag_fraction=bf,
        vps=vps,
        min_tp=min_tp,
        max_tp=max_tp,
        overwrite=True
        )
  classifiers[season] = clf
  classification_tasks[season] = classification_task

track_task(classification_tasks)

lc_classes_headers = [
                      '',
        'Irrigated Trees',
        'Irrigated Crops',
        'Forest',
        'Shrub',
        'Rainfed Agriculture',
        'Greenhouses',
        'Urban Fallow',
        'Water Bodies'
    ]

lc_classes_index = [
        'Irrigated Trees',
        'Irrigated Crops',
        'Forest',
        'Shrub',
        'Rainfed Agriculture',
        'Greenhouses',
        'Urban Fallow',
        'Water Bodies'
    ]

output.clear()

for clf in classifiers:
  conf_matrix = classifiers[clf].confusionMatrix()
  accuracy = conf_matrix.accuracy().getInfo()
  matrix = conf_matrix.getInfo()
  df = pd.DataFrame(matrix).drop(columns=[0], index=[0])
  df['Index'] = lc_classes_index
  df = df.set_index('Index').reset_index()

  fig = go.Figure(layout={'title':f'Confusion Matrix {clf.title()} Classifier. Accuracy: {round(accuracy * 100)}%'},
      data=[go.Table(
    header=dict(values=lc_classes_headers,
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=df.transpose().values.tolist(),
               fill_color='lavender',
               align='left'))
  ])

  fig['layout'].update(margin=dict(b=0), height=320)

  fig.show()

In [ ]:
# @markdown #<font color='yellow'> <== **Run this code block to visualize the results of the Random Forest classification** </font>

summer_clf = ee.Image(f'{results_folder}random_forest/cdc/ia_random_forest_{no_trees}tr_{vps}vps_{int(bf*100)}bf_cdc_summer_{year}')
winter_clf = ee.Image(f'{results_folder}random_forest/cdc/ia_random_forest_{no_trees}tr_{vps}vps_{int(bf*100)}bf_cdc_winter_{year}')

images_clf  = {
    f'RGB Composite of derived from satellite data for the summer of {year}':feature_data_summer.clip(aoi).getMapId(visualization.vis_params_rgb(max_val=.3, bands=['R_median', 'G_median', 'B_median'])),
    f'RGB Composite of derived from satellite data for the winter of {year}':feature_data_winter.clip(aoi).getMapId(visualization.vis_params_rgb(max_val=.3, bands=['R_median', 'G_median', 'B_median'])),
    'Summer RF Classification results': summer_clf.clip(aoi).getMapId(visualization.vis_params_cp('rf_all_classes', 0, 8, list(PALETTE_RF.values()))),
    'Winter RF Classification results': winter_clf.clip(aoi).getMapId(visualization.vis_params_cp('rf_all_classes', 0, 8, list(PALETTE_RF.values()))),
}

from google.colab.output import eval_js
eval_js('google.colab.output.setIframeHeight("1000")')

palette = PALETTE_RF.copy()
palette.pop('not_selected')

map = visualization.create_folium_map(images_clf, coords=[aoi_centroid[1], aoi_centroid[0]], zoom=12, height='100%')
map = visualization.create_categorical_legend(map, palette)
map 

# Step 4 - Generate the seasonal irrigation map

With the classification for the summer and winter completed, the next step is to combine the information of the irrigated areas during winter and summer into a single map. Run the code block below to create a map combining the irrigated areas, and to visualise the end result!

## Misclassifications

In the classification results we've found that during the winter season is is common for grass/shrub areas to be classified as irrigated trees, especially during wet winters winters. Furthermore, wetlands and permanenly irrigated grasslands such as golf courses and gardens also are likely to be classified as irrigated trees.    







In [ ]:
# @markdown <font color='yellow'> <== **Run this code block to combine irrigated areas for both the summer & winter into a single map** </font>
ia_summer = summer_clf.select('irrigated_area')
ia_winter = winter_clf.select('irrigated_area')
task = join_seasonal_irrigated_areas(
    ia_summer,
    ia_winter,
    f'cdc',
    year,
    aoi,
    overwrite=True,
    export_method='asset',
  )
track_task(task)

ia_year = ee.Image(f'{results_folder}irrigated_area/cdc/irrigated_areas_cdc_{year}').clip(aoi)

empty = ee.Image().byte();
outline = empty.paint(
  featureCollection=aoi,
  color=1,
  width=3.5
)

images_results = {
    f'RGB Composite of derived from satellite data for the summer of {year}':feature_data_summer.clip(aoi).getMapId(visualization.vis_params_rgb(max_val=.3, bands=['R_median', 'G_median', 'B_median'])),
    f'RGB Composite of derived from satellite data for the winter of {year}':feature_data_winter.clip(aoi).getMapId(visualization.vis_params_rgb(max_val=.3, bands=['R_median', 'G_median', 'B_median'])),
    'Area of Interest':outline.getMapId({'palette': 'FFFFFF'})
    }

total_irrigated_area =  calc_area(ia_year.select('ia_year').gt(0), aoi).getInfo()

legend = {}

for cl_val, cl in enumerate(PALETTE_IA.keys()):
  class_area =  calc_area(ia_year.select('ia_year').eq(cl_val), aoi).getInfo()
  legend[f"{cl} ({round(class_area)} ha.)"] = PALETTE_IA[cl]
  images_results[cl] = ia_year.mask(ia_year.eq(cl_val)).getMapId(visualization.vis_params_cp('ia_year', cl_val, cl_val, [PALETTE_IA[cl]]))

from google.colab.output import eval_js
eval_js('google.colab.output.setIframeHeight("1200")')

map = visualization.create_folium_map(images_results, coords=[aoi_centroid[1], aoi_centroid[0]], zoom=11, height='100%')
map = visualization.create_categorical_legend(map, legend)
map = visualization.create_hectares_label(map, round(total_irrigated_area), year)
output.clear()
map

In [ ]:
#@markdown # Export irrigated area map to Google drive
#@markdown ---
#@markdown To export the irrigated area map to your google drive as a GeoTiff run this code block. A CSV containing the classname and color per pixel value will also be exported to your drive.

file_name = "irrigated_area_map" #@param {type:"string"}
folder_name = "irrigated_areas_result" #@param {type:"string"}

crs = ia_year.select('ia_year').projection().getInfo()

task = ee.batch.Export.image.toDrive(
    ia_year.select('ia_year'),
    description=file_name,
    folder=folder_name,
    maxPixels=1000000000000,
    region=aoi_coordinates,
    crs=crs['crs'],
    crsTransform= crs['transform'],
    )

metadata_ia_map = ee.FeatureCollection([ee.Feature(None, {
    'class': key,
    'pixel_value':ind,
    'color':PALETTE_IA[key],
    'map':'seasonal irrigated area map'}) for ind, key in enumerate(PALETTE_IA.keys())]
)
    
task_metadata = ee.batch.Export.table.toDrive(
    metadata_ia_map, 
    description=f"{file_name}_metadata", 
    folder=folder_name,
    fileFormat="csv",
    selectors=['class', 'pixel_value', 'color']
  )

task.start()
task_metadata.start()

progress = track_task({
    'image_export':task,
    'metadata_export':task_metadata,
})

if progress:
  print(f'The classification result is now available on your drive: {task.status()["destination_uris"][0]}')